In [1]:
# https://stackoverflow.com/questions/66116155/how-to-tell-pytorch-which-cuda-version-to-take
# https://github.com/pytorch/pytorch/issues/75992

# Previous Versions of Pytorch avaiable here: https://pytorch.org/get-started/previous-versions/
#! pip install --upgrade torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0
#! pip install --upgrade torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0
#! pip install --upgrade torch==1.12.1 --extra-index-url https://download.pytorch.org/whl/cu113 #<--This version works for CUDA 11.4



In [2]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc


In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

In [4]:
torch.__version__
#'1.12.0+cu111'
#'1.9.0+cu111' This is the version by default


'1.12.1+cu113'

In [5]:
import tensorflow as tf
#physical_devices = tf.config.list_physical_devices('CPU')
physical_devices = tf.config.list_physical_devices('GPU')

In [6]:
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'


In [8]:
#test
x = torch.rand(5, 3, 
               device=device#'cpu'
               )
print(x)

tensor([[0.3245, 0.7223, 0.6211],
        [0.8187, 0.3495, 0.1357],
        [0.0968, 0.1712, 0.3243],
        [0.7788, 0.4892, 0.3519],
        [0.3324, 0.2398, 0.7875]])


In [9]:
x.get_device()

-1

In [10]:
#Deleting tensor to free memory
del x
torch.cuda.empty_cache()

In [11]:
torch.cuda.memory_allocated()

0

In [12]:
#! conda install pytorch torchvision torchaudio cudatoolkit=10.1 -c pytorch
#! pip3 install torch==1.7.0 torchvision==0.8.1 -f https://download.pytorch.org/whl/cu101/torch_stable.html
#! pip install tensorflow
! nvidia-smi

Fri Feb 17 22:59:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-PCI...  Off  | 00000000:61:00.0 Off |                    0 |
| N/A   31C    P0    35W / 250W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Logits Extractor
>
> Extracting Tensor Logits from a given Neural Code Model @danaderp
>

In [13]:
#Available Datasets
# Case1: codesearch_tesbed_EleutherAI-gpt-neo-125M_10000 for the model 'EleutherAI/gpt-neo-125M' 
# Case2: codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000 for the model 'EleutherAI/gpt-neo-2.7B' <-- MEMORY CONSTRAINTS
# Case3: codesearch_tesbed_EleutherAI-gpt-neo-1.3B_10000 for the model 'EleutherAI/gpt-neo-1.3B'
def params(): 
    return {
            'big_table_path' : '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv',
            'hf_model' : 'EleutherAI/gpt-neo-2.7B',
            'model_name': 'EleutherAI-gpt-neo-2.7B_10000_',
            'callbacks' : '../data/callbacks',
            'batch': 1 
}

In [14]:
#pwd
parameters = params()
parameters['big_table_path']

'../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv'

# Data Upload

In [15]:
data_pd = pd.read_csv( 
                      parameters['big_table_path'] , 
                      index_col=0
            )

In [16]:
data_pd.describe()

,model_total_input_ids
count,10000.000000
mean,417.657300
std,362.953784
min,46.000000
25%,169.000000
50%,287.000000
75%,541.000000
max,2044.000000


In [17]:
data_pd.head(5)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids
0,"def encode_images(format_dict):\n """"""b64-en...","[('def', 'def', 'function_definition'), ('enco...","[(4299, 'def', 'function_definition'), (37773,...","[4299, 37773, 62, 17566, 7, 18982, 62, 11600, ...",355
1,"def _process_execute_error(self, msg):\n ...","[('def', 'def', 'function_definition'), ('_pro...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 14681, 62, 41049, 62, 18224, 7, 9...",398
2,"def long_file(data_file, dataformat, sample_li...","[('def', 'def', 'function_definition'), ('long...","[(4299, 'def', 'function_definition'), (890, '...","[4299, 890, 62, 7753, 7, 7890, 62, 7753, 11, 1...",1921
3,"def snapshot(self,\n label,\n ...","[('def', 'def', 'function_definition'), ('snap...","[(4299, 'def', 'function_definition'), (27479,...","[4299, 27479, 7, 944, 11, 198, 220, 220, 220, ...",801
4,def unload(self):\n '''Releases rendere...,"[('def', 'def', 'function_definition'), ('unlo...","[(4299, 'def', 'function_definition'), (555, '...","[4299, 555, 2220, 7, 944, 2599, 198, 220, 220,...",79


# Model Upload

In [18]:
#! pip install transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [19]:
#pip list
#! pip install git+https://github.com/huggingface/transfomers.git


In [20]:
#tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-2B-mono")
#tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
#generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')
#model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")


def testing_1():
    # This is for 'EleutherAI/gpt-neo-125M'
    tokenizer = AutoTokenizer.from_pretrained(parameters['hf_model'])
    generator = pipeline(
        'text-generation', 
        model= parameters['hf_model'] )
    
    #TEST: Example 1: generation
    generator(
        "EleutherAI has", 
        do_sample=True, 
        max_new_tokens=20, 
        pad_token_id = tokenizer.eos_token_id
        )
    
    #TEST: Example 2: generation

    generator( 
        data_pd.whole_func_string.values[0][:100], #<-- Code data
        do_sample=True, 
        max_new_tokens=20,
        pad_token_id = tokenizer.eos_token_id 
        )



# Extracting Logits From a Given Model

In [21]:
#This code works for GPT-Neo
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
model = GPTNeoForCausalLM.from_pretrained( parameters['hf_model'] )
tokenizer = GPT2Tokenizer.from_pretrained( parameters['hf_model'] )

In [22]:
#tokenizer.get_vocab()

In [23]:
#len( tokenizer.get_vocab() ) #todo an assert

In [24]:
#prompts = data_pd.whole_func_string.values[:2]
prompts = data_pd.whole_func_string.values

In [25]:
len(prompts)

10000

In [26]:
#Making space for the tensors
input_ids_list = tokenizer.batch_encode_plus( prompts ) #<-- Do not return as a Tensor
#input_ids_list = tokenizer.batch_encode_plus( prompts , return_tensors="pt") #<-- "pt" returns as a Tensor

In [27]:
#Casting Integers to Tensor Integers. Make sure the tesor is created in a device
#We ignored the parameter attention_mask since we are not using masking here [https://huggingface.co/transformers/v4.10.1/glossary.html#attention-mask]

#input_ids_list = [torch.Tensor( np.array( input_ids ) ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
#input_ids_list = [ input_ids for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]
input_ids_list = [torch.tensor(  input_ids, dtype = torch.int, device=device ) for input_ids in input_ids_list.input_ids if len(input_ids) <= 2048]

In [28]:
#It should be same size
assert len(input_ids_list) == len(prompts)

In [29]:
data_pd.model_input_ids.values[0]

'[4299, 37773, 62, 17566, 7, 18982, 62, 11600, 2599, 198, 220, 220, 220, 37227, 65, 2414, 12, 12685, 4147, 4263, 287, 257, 3359, 12984, 5794, 8633, 198, 220, 220, 220, 220, 198, 220, 220, 220, 8673, 428, 815, 307, 12118, 287, 33918, 62, 27773, 2346, 30, 198, 220, 220, 220, 220, 198, 220, 220, 220, 40117, 198, 220, 220, 220, 24200, 438, 198, 220, 220, 220, 220, 198, 220, 220, 220, 5794, 62, 11600, 1058, 8633, 198, 220, 220, 220, 220, 220, 220, 220, 317, 22155, 286, 3359, 1366, 1994, 276, 416, 285, 524, 12, 4906, 198, 220, 220, 220, 220, 198, 220, 220, 220, 16409, 198, 220, 220, 220, 35656, 198, 220, 220, 220, 220, 198, 220, 220, 220, 5794, 62, 11600, 1058, 8633, 198, 220, 220, 220, 220, 220, 220, 220, 317, 4866, 286, 262, 976, 22155, 11, 198, 220, 220, 220, 220, 220, 220, 220, 475, 13934, 2939, 1366, 19203, 9060, 14, 11134, 6, 393, 705, 9060, 14, 73, 22071, 11537, 198, 220, 220, 220, 220, 220, 220, 220, 318, 2779, 2414, 12, 12685, 9043, 13, 198, 220, 220, 220, 220, 198, 220, 220, 220, 3

In [30]:
input_ids_list[0]

tensor([ 4299, 37773,    62, 17566,     7, 18982,    62, 11600,  2599,   198,
          220,   220,   220, 37227,    65,  2414,    12, 12685,  4147,  4263,
          287,   257,  3359, 12984,  5794,  8633,   198,   220,   220,   220,
          220,   198,   220,   220,   220,  8673,   428,   815,   307, 12118,
          287, 33918,    62, 27773,  2346,    30,   198,   220,   220,   220,
          220,   198,   220,   220,   220, 40117,   198,   220,   220,   220,
        24200,   438,   198,   220,   220,   220,   220,   198,   220,   220,
          220,  5794,    62, 11600,  1058,  8633,   198,   220,   220,   220,
          220,   220,   220,   220,   317, 22155,   286,  3359,  1366,  1994,
          276,   416,   285,   524,    12,  4906,   198,   220,   220,   220,
          220,   198,   220,   220,   220, 16409,   198,   220,   220,   220,
        35656,   198,   220,   220,   220,   220,   198,   220,   220,   220,
         5794,    62, 11600,  1058,  8633,   198,   220,   220, 

In [31]:
# tokenizer.decode( input_ids_list[0] ) #Decoding IDs

In [32]:
#model.to( 'cpu' ) #WARNING, 
model.to( device ) #WARNING, Verify the device before assigning to memory

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2560)
    (wpe): Embedding(2048, 2560)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
            (out_proj): Linear(in_features=2560, out_features=2560, bias=True)
          )
        )
        (ln_2): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2560, out_features=10240, bias=True)
          (c_proj): Linear

In [33]:
parameters #Verification Point of Parameters

{'big_table_path': '../data/concept_tables/codesearch_tesbed_EleutherAI-gpt-neo-2.7B_10000.csv',
 'hf_model': 'EleutherAI/gpt-neo-2.7B',
 'model_name': 'EleutherAI-gpt-neo-2.7B_10000_',
 'callbacks': '../data/callbacks',
 'batch': 1}

In [34]:
def logit_extractor(batch, input, from_index=0):
    """
    Output is the class CausalLMOutputWithPast (https://huggingface.co/transformers/v4.10.1/main_classes/output.html?highlight=causallmoutputwithpast)"
    logits (torch.FloatTensor of shape (batch_size, sequence_length, config.vocab_size)) – Prediction scores of the language modeling head (scores for each vocabulary token before SoftMax).
    The expression i.type(torch.LongTensor).to(device) is for casting labels for the loss
    """
    #Output is in CausalLMOutputWithPast

    for idx, n in enumerate( range( from_index, len(input), batch) ):
        output = [ model( 
            input_ids = i, 
            labels = i.type(torch.LongTensor).to(device) 
            ) for i in input[n:n+batch] ] #Labels must be provided to compute loss
    
        output_logits = [ o.logits.detach().to('cpu').numpy() for o in output ]  #Logits Extraction
        output_loss = np.array([ o.loss.detach().to('cpu').numpy() for o in output ])  #Language modeling loss (for next-token prediction).

        #Saving Callbacks
        current_batch = idx + (from_index//batch)
        for jdx, o_logits in enumerate( output_logits ):
            np.save( parameters['callbacks']+ '/'+parameters['model_name']  + f'_logits_tensor[{jdx+n}]_batch[{current_batch}].npy', o_logits)
        np.save( parameters['callbacks']+ '/'+parameters['model_name']+f'_loss_batch[{current_batch}].npy', output_loss)
        
        print(f"Batch [{current_batch}] Completed")
        
        #Memory Released
        for out in output:
            del out.logits
            torch.cuda.empty_cache()
            del out.loss
            torch.cuda.empty_cache()
        for out in output_logits:
            del out
            torch.cuda.empty_cache()
        for out in output_loss:
            del out
            torch.cuda.empty_cache()
            

    pass

In [35]:
#logit_extractor(batch =2, input= input_ids_list[:2]) #<---- [WARNING TIME AND MEMORY CONSUMING]

In [36]:
#output_logits = np.load('../data/callbacks/logits_tensor[0]_batch[0].npy')

In [37]:
#assert output_logits.shape[0] == len(input_ids_list[0])

In [38]:
#output_loss = np.load('../data/callbacks/loss_batch[0].npy')

In [39]:
#output_loss

In [40]:
## ACTUAL EXPERIMENT
## TIME AND MEMORY CONSUMING
logit_extractor(batch = 1, input= input_ids_list, from_index=2)

Batch [2] Completed
Batch [3] Completed
Batch [4] Completed
Batch [5] Completed
Batch [6] Completed
Batch [7] Completed
Batch [8] Completed
Batch [9] Completed
Batch [10] Completed
Batch [11] Completed
Batch [12] Completed
Batch [13] Completed
Batch [14] Completed
Batch [15] Completed
Batch [16] Completed
Batch [17] Completed
Batch [18] Completed
Batch [19] Completed
Batch [20] Completed
Batch [21] Completed
Batch [22] Completed
Batch [23] Completed
Batch [24] Completed
Batch [25] Completed
Batch [26] Completed
Batch [27] Completed
Batch [28] Completed
Batch [29] Completed
Batch [30] Completed
Batch [31] Completed
Batch [32] Completed
Batch [33] Completed
Batch [34] Completed
Batch [35] Completed
Batch [36] Completed
Batch [37] Completed
Batch [38] Completed
Batch [39] Completed
Batch [40] Completed
Batch [41] Completed
Batch [42] Completed
Batch [43] Completed
Batch [44] Completed
Batch [45] Completed
Batch [46] Completed
Batch [47] Completed
Batch [48] Completed
Batch [49] Completed
